# 01_train.ipynb
Entrenamiento rápido (ResNet18).

In [ ]:

!pip install torch torchvision torchaudio matplotlib scikit-learn google-cloud-storage -q
import json, numpy as np, matplotlib.pyplot as plt
import torch, torchvision
from torchvision import datasets, transforms, models
from torch import nn, optim
from sklearn.metrics import classification_report, confusion_matrix
import os
os.makedirs("models", exist_ok=True)

data_dir = "./data"  # Estructura: data/train/<clase>, data/val/<clase>
tfms_train = transforms.Compose([
    transforms.Resize((224,224)), transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
tfms_val = transforms.Compose([
    transforms.Resize((224,224)), transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_ds = datasets.ImageFolder(f"{data_dir}/train", tfms_train)
val_ds   = datasets.ImageFolder(f"{data_dir}/val",   tfms_val)
class_names = train_ds.classes
json.dump({i:c for i,c in enumerate(class_names)}, open("models/class_map.json","w"))
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_dl   = torch.utils.data.DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(class_names)); model = model.to(device)
criterion = nn.CrossEntropyLoss(); optimizer = optim.Adam(model.parameters(), lr=1e-3)

def evaluate():
    model.eval(); y_true=[]; y_pred=[]
    with torch.no_grad():
        for x,y in val_dl:
            x,y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            y_true += y.tolist(); y_pred += preds.tolist()
    return y_true, y_pred

best_acc, best_state = 0.0, None
for epoch in range(5):
    model.train()
    for x,y in train_dl:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y); loss.backward(); optimizer.step()
    yt, yp = evaluate()
    import numpy as np
    acc = (np.array(yt)==np.array(yp)).mean()
    print(f"Epoch {epoch+1}: val_acc={acc:.3f}")
    if acc>best_acc: best_acc, best_state = acc, model.state_dict().copy()
model.load_state_dict(best_state)
torch.save({"state_dict": model.state_dict(), "classes": class_names}, "models/model.pt")

print(classification_report(yt, yp, target_names=class_names))
cm = confusion_matrix(yt, yp); plt.imshow(cm); plt.title("Matriz de confusión"); plt.show()
